In [21]:
%matplotlib inline
import os
import sys
import string
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import time
import csv

plt.rcParams['figure.figsize'] = (10,7)

import subprocess
import glob
import itertools

In [22]:
#!pip freeze

In [23]:
##########################  
# Essa funcao é definitiva, ela das três formas é a mais eficiente, 
def novo_shuffle(numero_estados, tamanho_espaco_busca):
    espaco_busca = []
    for i in range (2**tamanho_espaco_busca):
        espaco_busca.append(bin(i)[2:].zfill(tamanho_espaco_busca))
    lista_remover = list(espaco_busca)
    nova_atrib = []
    for i in range(numero_estados):
        valor = random.choice(lista_remover)
        lista_remover.remove(valor)
        nova_atrib.append(valor)
    
    return nova_atrib

In [24]:
def correspondente(palavra1, palavra2, tam_space):
    espaco_busca = []
    for i in range (2**tam_space):
        espaco_busca.append(bin(i)[2:].zfill(tam_space))
    espaco_aleatorio = palavra2

    for i in range(len(espaco_busca)):
        if palavra1 == espaco_busca[i]:
            return str(espaco_aleatorio[i])

In [25]:
def simplifica_maquina(filename):
    teste = subprocess.run(['./espresso',filename],stdout=subprocess.PIPE,universal_newlines=True)
    saida = teste.stdout
    a = open(filename,'w')
    a.write(saida)
    a.close()

In [26]:
def leitura(filename):
    nova = open(filename,'r')
    line_char = nova.readlines()
    custo_nova = int(line_char[2][3:])
    nova.close()
    return custo_nova

In [27]:
def troca_maquina(newfilename, bestfilename):
    nova = open(newfilename,'r')
    melhor = open(bestfilename,'w')
    nova_lista = nova.readlines()
    for i in range(len(nova_lista)):
        melhor.write(str(nova_lista[i]))
    nova.close()
    melhor.close()

In [28]:
def calculo_custo(filename, entrada_input, saida_output, numero_estados):
    transicoes_nova = leitura(filename)
    area = transicoes_nova*(2*entrada_input + 3*(np.log2(numero_estados)) + saida_output)
    return area

In [29]:
def resfriamento_simulado(casefilename, temperatura, vizinhos):
    #####################################################################
    # Faz a leitura do arquivo KISS2 e prepara as variáveis globais
    l = []
    entrada = []
    lista_entradas = []
    saida = []
    lista_saidas = []

    estados = []

    atual = []
    atual_inteiro = []
    atual_convertido = []

    proximo = []
    proximo_inteiro = []
    proximo_convertido = []

    espaco_aleatorio = []

    comprimento_input = 0
    comprimento_output = 0
    line_char = []
    msf_pronta = []

    tamanho_espaco_busca = 0
    
    temp = open('msfs/' + casefilename +'.kiss2')

    line_char = temp.readlines()
    
    estados = line_char[4:5]
    estados_convertido = int(estados[0].split()[1])

    #define o tamanho do espaço de busca basedado no valor entregue pelo arquivo

    tamanho_espaco_busca = math.ceil(np.log2(estados_convertido))

    entrada_input = int(line_char[1][3:])
    saida_output = int(line_char[2][3:])
    numero_estados = int(line_char[4][3:])

    l = line_char[5:]
    
    for i in range(len(l)):
        entrada = l[i].split()[0]
        lista_entradas.append(entrada)
    
        atual = l[i].split()[1]
        proximo = l[i].split()[2]
        saida = l[i].split()[3]
        lista_saidas.append(saida)
    
        atual_inteiro = int(''.join([i for i in atual if i.isdigit()]))
        proximo_inteiro = int(''.join([i for i in proximo if i.isdigit()]))

        atual_convertido.append(bin(atual_inteiro)[2:].zfill(tamanho_espaco_busca))
        proximo_convertido.append(bin(proximo_inteiro)[2:].zfill(tamanho_espaco_busca))
        linha = str(lista_entradas[i])+str(atual_convertido[i])+' '+str(proximo_convertido[i])+str(lista_saidas[i])
        msf_pronta.append(linha)

    comprimento_input = str(len(lista_entradas[0])+len(atual_convertido[0]))
    comprimento_output = str(len(proximo_convertido[0])+len(lista_saidas[0]))

    d = open('melhor.pla','w')
    #d.write('\n')
    d.write('.i '+comprimento_input+'\n')
    d.write('.o '+comprimento_output+'\n')
    d.write(line_char[3])
    
    size = len(msf_pronta)

    for i in range(size):
        d.write(msf_pronta[i]+'\n')

    d.close()
    #######################################################################
    #reseta_msf('melhor.pla')
    temperatura_inicial = temperatura
    temperatura_final = 1

    simplifica_maquina('melhor.pla')
    custo_melhor = calculo_custo('melhor.pla', entrada_input, saida_output, numero_estados)
    custo_init = custo_melhor
    solucao_global = custo_init
    historico_temperatura = [temperatura]
    historico = [custo_melhor]
    while(temperatura > temperatura_final):
        for i in range(vizinhos):
            #custo_melhor = calculo_custo('melhor.pla', entrada_input, saida_output, numero_estados)
            #################################################################################
            # Cria nova MSF
            novo_atual = []
            novo_proximo = []
            nova_msf = []
    
            espaco_aleatorio = list(novo_shuffle(numero_estados, tamanho_espaco_busca))
    
            for i in range(len(l)):
                novo_atual.append(correspondente(atual_convertido[i],espaco_aleatorio, tamanho_espaco_busca))
                novo_proximo.append(correspondente(proximo_convertido[i],espaco_aleatorio, tamanho_espaco_busca))
                nova_atrib = str(lista_entradas[i])+str(novo_atual[i])+' '+str(novo_proximo[i])+str(lista_saidas[i])
                nova_msf.append(nova_atrib)
        
            d = open('nova.pla','w')
            d.write('.i ' + str(comprimento_input) + '\n')
            d.write('.o ' + str(comprimento_output) + '\n')
            d.write(line_char[3])

            for i in range(len(nova_msf)):
                d.write(nova_msf[i]+'\n')
            d.close()
            ##########################################################################
            simplifica_maquina('nova.pla')
            custo_nova = calculo_custo('nova.pla', entrada_input, saida_output, numero_estados)
            probabilidade = np.random.random_sample()

            expo_value = np.exp(-(custo_nova - custo_melhor)/temperatura)
            
            
            
            if (custo_nova < custo_melhor):
                troca_maquina('nova.pla', 'melhor.pla')
                custo_melhor = custo_nova
            else:
                #print("Expo_value com custo pior: {:.2f}".format(expo_value))
                if (probabilidade < expo_value):
                    troca_maquina('nova.pla', 'melhor.pla')
                    custo_melhor = custo_nova
                    
            if (custo_melhor < solucao_global):
                solucao_global = custo_melhor
        
        if temperatura > temperatura_inicial/2:
            temperatura = temperatura - 1.2
        else:
            temperatura = temperatura - 0.8

        historico.append(solucao_global)
        historico_temperatura.append(temperatura)
        
    return custo_init, solucao_global, historico, historico_temperatura

In [ ]:
kissfiles = []
for file in glob.glob("msfs/*.kiss2"):
    name = file.split('/')
    name = name[1].split('.')
    kissfiles.append(name[0])
kissfiles.sort()
#kissfiles = ['bbtas']    
print(kissfiles)

with open('resultados.csv', mode='w') as csv_file:
    fieldnames = ['Caso', 'Custo Inicial 0', 'Custo Final 0', 'Percentual 0', 'Tempo (s) 0',
                 'Custo Inicial 1', 'Custo Final 1', 'Percentual 1', 'Tempo (s) 1',
                 'Custo Inicial 2', 'Custo Final 2', 'Percentual 2', 'Tempo (s) 2',
                 'Custo Inicial 3', 'Custo Final 3', 'Percentual 3', 'Tempo (s) 3',
                 'Custo Inicial 4', 'Custo Final 4', 'Percentual 4', 'Tempo (s) 4',
                 'Custo Inicial 5', 'Custo Final 5', 'Percentual 5', 'Tempo (s) 5',
                 'Custo Inicial 6', 'Custo Final 6', 'Percentual 6', 'Tempo (s) 6',
                 'Custo Inicial 7', 'Custo Final 7', 'Percentual 7', 'Tempo (s) 7',
                 'Custo Inicial 8', 'Custo Final 8', 'Percentual 8', 'Tempo (s) 8',
                 'Custo Inicial 9', 'Custo Final 9', 'Percentual 9', 'Tempo (s) 9']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    
    markers = itertools.cycmle(['o','v','^','<','>','s','p','*','h','+','x','d','1','2','3','4','8'])
    for k in range(10): 
      plt.cla()  
      for name in kissfiles:    
        temperatura = 100
        custo_init = 0
        custo_final = 0
        valores = []
        temperatura_iter = []
    
        tic = time.perf_counter()
        custo_init, custo_final, valores, temperatura_iter = resfriamento_simulado(name, temperatura, 100)
        toc = time.perf_counter()
        tempo = str(toc-tic)
        
        print("Melhor custo:",custo_final)
        print("Tempo de execucao:",toc-tic)
    
        writer.writerow({'Caso': name, 
                         'Custo Inicial ' + str(k): str(custo_init), 
                         'Custo Final ' + str(k): str(custo_final),
                         'Percentual ' + str(k): str(1 - (custo_final/custo_init)),
                         'Tempo (s) ' + str(k): tempo
                        })
        
        for i in range(len(valores)):
            valores[i] = valores[i] / custo_init
            temperatura_iter[i] = temperatura_iter[i] / temperatura
        
        
        plt.xlabel('Iterations')  # Add an x-label to the axes.
        plt.ylabel('Cost')
        plt.plot(valores, marker=next(markers), label=name)
        
        print('Run ' + str(k) + ', ' + name + ': ok')
         
    plt.legend(loc='upper center', ncol=6, bbox_to_anchor=(0., 1.05, 1., .105))
    plt.figure(1)    
    plt.savefig('resultados.png')